In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\AI-ML\campus_x\INTERVIEW_PREP\Onboarding\data\main.csv")
df.head(2)

,id,completion_date,partner_name,deal_type,review_type,escalation_type,EDD_reviewer,EDD_measures,timestamp
0,1,01-01-2023,Partner A,imt,fresh_onboarding,other_red_flags,neelima_routhu,document_review,01-01-2023 10:00
1,10,10-01-2023,Partner J,payments,periodic_review,other_red_flags,rohan_vazapully,aml_review,10-01-2023 19:00


In [ ]:
pd.pivot_table(data = df,
               index= "EDD_reviewer", 
               columns= ["escalation_type"],
               values= "escalation_type",
               aggfunc= "count"
)

In [7]:
# Aggregating the counts of escalation types and EDD measures per reviewer
agg_data = df.groupby('EDD_reviewer').agg(
    other_red_flags=('escalation_type', lambda x: (x == 'other_red_flags').sum()),
    adverse_media=('escalation_type', lambda x: (x == 'adverse_media').sum()),
    high_risk=('escalation_type', lambda x: (x == 'high_risk').sum()),
    pep_association=('escalation_type', lambda x: (x == 'pep_association').sum()),
    country_risk=('escalation_type', lambda x: (x == 'country_risk').sum()),
    complex_ownership=('escalation_type', lambda x: (x == 'complex_ownership').sum()),
    young_company=('escalation_type', lambda x: (x == 'young_company').sum()),
    medium_risk=('escalation_type', lambda x: (x == 'medium_risk').sum()),
    feedback_review_esc=('escalation_type', lambda x: (x == 'feedback_review').sum()),
    document_review=('EDD_measures', lambda x: (x == 'document_review').sum()),
    aml_review=('EDD_measures', lambda x: (x == 'aml_review').sum()),
    audit_review=('EDD_measures', lambda x: (x == 'audit_review').sum()),
    feedback_review_meas=('EDD_measures', lambda x: (x == 'feedback_review').sum())
).reset_index()

agg_data


,EDD_reviewer,other_red_flags,adverse_media,high_risk,pep_association,country_risk,complex_ownership,young_company,medium_risk,feedback_review_esc,document_review,aml_review,audit_review,feedback_review_meas
0,francis_xavier,0,2,1,2,3,2,3,2,2,5,6,2,3
1,hityshi,1,2,2,2,2,1,2,0,1,3,3,5,2
2,laura_castillo,3,3,1,2,0,2,1,2,2,3,5,2,6
3,moustapha,2,1,3,1,4,0,2,1,2,5,3,4,3
4,neelima_routhu,4,1,3,3,2,3,1,2,0,10,2,3,5
5,rahil_fw,2,2,0,2,0,2,2,3,2,2,4,4,5
6,rohan_vazapully,2,1,2,2,2,1,2,2,1,3,4,6,2


In [8]:
# Define weights for each escalation type and EDD measure
weights = {
    'other_red_flags': 1,
    'adverse_media': 3,
    'high_risk': 5,
    'pep_association': 3,
    'country_risk': 1,
    'complex_ownership': 1,
    'young_company': 1,
    'medium_risk': 4,
    'combined_reviews': 2  # Combined weight for document review, aml review, audit review, and feedback review
}

# Add the "Total Working Hours" column with a constant value of 40
agg_data['Total Working Hours'] = 40

# Calculate the combined reviews
agg_data['combined_reviews'] = (
    agg_data['document_review'] +
    agg_data['aml_review'] +
    agg_data['audit_review'] +
    agg_data['feedback_review_esc'] +
    agg_data['feedback_review_meas']
)

# Calculate the "Total SLA period" for each row
agg_data['Total SLA period'] = (
    agg_data['other_red_flags'] * weights['other_red_flags'] +
    agg_data['adverse_media'] * weights['adverse_media'] +
    agg_data['high_risk'] * weights['high_risk'] +
    agg_data['pep_association'] * weights['pep_association'] +
    agg_data['country_risk'] * weights['country_risk'] +
    agg_data['complex_ownership'] * weights['complex_ownership'] +
    agg_data['young_company'] * weights['young_company'] +
    agg_data['medium_risk'] * weights['medium_risk'] +
    agg_data['combined_reviews'] * weights['combined_reviews']
)

# Calculate the "Difference" column
agg_data['Difference'] = agg_data['Total Working Hours'] - agg_data['Total SLA period']

agg_data


,EDD_reviewer,other_red_flags,adverse_media,high_risk,pep_association,country_risk,complex_ownership,young_company,medium_risk,feedback_review_esc,document_review,aml_review,audit_review,feedback_review_meas,Total Working Hours,combined_reviews,Total SLA period,Difference
0,francis_xavier,0,2,1,2,3,2,3,2,2,5,6,2,3,40,18,69,-29
1,hityshi,1,2,2,2,2,1,2,0,1,3,3,5,2,40,14,56,-16
2,laura_castillo,3,3,1,2,0,2,1,2,2,3,5,2,6,40,18,70,-30
3,moustapha,2,1,3,1,4,0,2,1,2,5,3,4,3,40,17,67,-27
4,neelima_routhu,4,1,3,3,2,3,1,2,0,10,2,3,5,40,20,85,-45
5,rahil_fw,2,2,0,2,0,2,2,3,2,2,4,4,5,40,17,64,-24
6,rohan_vazapully,2,1,2,2,2,1,2,2,1,3,4,6,2,40,16,66,-26
